In [43]:
import pandas as pd
from glob import glob
import json
import os
import numpy as np

In [198]:
# jsons = glob("data/simulation/*/result.json")
data = []
for i in range(0, 1000):
    if os.path.exists(f"data/simulation/virtual customer {i}/result.json"):
        with open(f"data/simulation/virtual customer {i}/result.json") as f:
            s = json.load(f)
            s['succeed'] = True
            s['group'] = 'T1' if i  % 2 == 0 else 'C'
            if 'price' in s:
                price = s["price"].replace("$", "").replace("\n", "").replace(",", "")
                if price.count(".") != 1:
                    print(price)
                    price = price[: len(price)//2]
                s["price"] = float(price)
            else:
                s['price'] = 0
            if os.path.exists(f"data/simulation/virtual customer {i}/action_trace.txt"):
                s['action'] = [json.loads(x) for x in open(f"data/simulation/virtual customer {i}/action_trace.txt").readlines()]
                s['flatten_action'] = [item for sublist in s['action'] for item in sublist]
                s['len_action'] = sum([len(x) for x in s['action']])
                s['num_click'] = sum([x['type'] == 'click' for x in s['flatten_action']])
                s['num_search'] = sum([x['type'] == 'type_and_submit' for x in s['flatten_action']])
                s["num_click_refinements"] = sum(
                    [
                        x["type"] == "click" and x["name"].startswith("refinements")
                        for x in s["flatten_action"]
                    ]
                )
                s["num_click_search_results"] = sum(
                    [
                        x["type"] == "click" and x["name"].startswith("search_results")
                        for x in s["flatten_action"]
                    ]
                )
                s["action_valid"] = sum(
                    [
                        # "type": "click", "name": "add_to_cart.buy_now"
                        x["type"] == "click" and x["name"] == "add_to_cart.buy_now"
                        for x in s["flatten_action"]
                    ]
                ) == 1
            persona = json.load(open(f"data/personas/json/virtual customer {i}.json"))
            s['persona'] = persona
            s['gender'] = persona['gender']
            s['income'] = persona['income']
            s['age'] = persona['age']
            data.append(s)
    else:
        data.append(
            {
                "succeed": False,
                "group": "T1" if i % 2 == 0 else "C",
                "price": 0,
                "terminated": False,
            }
        )

449.99449.99


In [232]:
df = pd.DataFrame(data)
df.loc[df["terminated"].isnull(), "terminated"] = False
df['terminated'] = df['terminated'].astype(bool)
df['succeed'] = df['succeed'].astype(bool)
df['error'] = (~df['terminated']) & (df['price'] == 0)

In [98]:
print(sum(df[df["group"] == "T1"]["terminated"]))

78


In [64]:
print(sum(df[df["group"] == "C"]["terminated"]))

55


In [65]:
print(sum(df[df["group"] == "T1"]["price"]))


27571.850000000082


In [67]:
print(sum(df[df["group"] == "C"]["price"]))


31986.530000000155


In [70]:
df.to_csv("data/simulation/result.csv")

In [76]:
from scipy.stats import ttest_ind


In [77]:
ttest_ind(df[df["group"] == "T1"]["price"], df[df["group"] == "C"]["price"])

TtestResult(statistic=np.float64(-0.5915668796912231), pvalue=np.float64(0.5542746276335935), df=np.float64(998.0))

In [78]:
print(sum(df[df["group"] == "C"]["price"] / 500), sum(df[df["group"] == "T1"]["price"] / 500))

63.97306 55.143700000000045


In [79]:
# variance
print(np.var(df[df["group"] == "C"]["price"]))

87279.3621740364


In [85]:
ttest_ind(
    df[df["group"] == "T1"]["terminated"].astype(int),
    df[df["group"] == "C"]["terminated"].astype(int),
)


TtestResult(statistic=np.float64(2.144648942917885), pvalue=np.float64(0.03222184097243871), df=np.float64(998.0))

In [91]:
# chi square
from scipy.stats import chisquare

chisquare(
    [sum(df[df["group"] == "C"]["terminated"].astype(int)), 500-sum(df[df["group"] == "C"]["terminated"].astype(int))],
    [sum(df[df["group"] == "T1"]["terminated"].astype(int)), 500-sum(df[df["group"] == "T1"]["terminated"].astype(int))],
)

Power_divergenceResult(statistic=np.float64(8.03560578442095), pvalue=np.float64(0.004586666814499163))

(78, 422)

In [92]:
"123".startswith("refinements")

False

In [99]:
df['action_valid'].sum()

814

In [109]:
filtered_df = df[df["action_valid"] == True]

In [104]:
filtered_df[filtered_df["group"] == "T1"]["num_click"].mean()

np.float64(3.41044776119403)

In [105]:
filtered_df[filtered_df["group"] == "C"]["num_click"].mean()


np.float64(3.3592233009708736)

In [106]:
filtered_df[filtered_df["group"] == "T1"]["num_click_refinements"].mean()


np.float64(0.5746268656716418)

In [108]:
filtered_df[filtered_df["group"] == "C"]["num_click_refinements"].mean()

np.float64(0.5194174757281553)

In [110]:
print(filtered_df[filtered_df["group"] == "T1"]["num_search"].mean())
print(filtered_df[filtered_df["group"] == "C"]["num_search"].mean())

1.1069651741293531
1.1529126213592233


In [111]:
print(filtered_df[filtered_df["group"] == "T1"]["num_click_search_results"].mean())
print(filtered_df[filtered_df["group"] == "C"]["num_click_search_results"].mean())


1.6467661691542288
1.6674757281553398


In [112]:
df.to_csv("data/result.csv")

In [120]:
# chi square
from scipy.stats import chisquare

chisquare(
    [
        sum(df[df["group"] == "C"]["price"].astype(bool)),
        500 - sum(df[df["group"] == "C"]["price"].astype(bool)),
    ],
    [
        sum(df[df["group"] == "T1"]["price"].astype(bool)),
        500 - sum(df[df["group"] == "T1"]["price"].astype(bool)),
    ],
)


Power_divergenceResult(statistic=np.float64(1.2790667928679236), pvalue=np.float64(0.25807262148304516))

In [116]:
sum(df[df["group"] == "T1"]["price"].astype(bool))

403

In [119]:
500 - sum(df[df["group"] == "T1"]["terminated"].astype(bool))


422

In [121]:
sum(df[df["group"] == "C"]["price"].astype(bool))


413

In [122]:
500 - sum(df[df["group"] == "C"]["terminated"].astype(bool))


445

In [124]:
ttest_ind(
    np.log(df[df["group"] == "T1"]["price"] + 1), np.log(df[df["group"] == "C"]["price"]+ 1)
)


TtestResult(statistic=np.float64(-0.22782898457865544), pvalue=np.float64(0.8198258595074086), df=np.float64(998.0))

In [125]:
np.log(df[df["group"] == "T1"]["price"] + 1)


0      3.526066
2      3.610648
4      4.262539
6      3.044046
8      0.000000
         ...   
990    0.000000
992    1.603420
994    4.730480
996    2.861057
998    2.102914
Name: price, Length: 500, dtype: float64

In [126]:
df[df["group"] == "T1"]["price"]


0       32.99
2       35.99
4       69.99
6       19.99
8        0.00
        ...  
990      0.00
992      3.97
994    112.35
996     16.48
998      7.19
Name: price, Length: 500, dtype: float64

In [128]:
df[df["age"] == 18]["price"]


70       3.57
249      0.00
287     78.98
304     12.99
352      5.88
377     20.99
439    159.99
656      6.99
682      0.00
726     21.67
750      7.99
833    104.95
994    112.35
Name: price, dtype: float64

In [129]:
# corelation between age and price, chi square
# and maybe income / gender, ops in each group, may not significant



In [234]:
df.to_excel("data/result.xlsx")

In [154]:
print(sum(df[df["group"] == "C"]["error"]))
print(sum(df[df["group"] == "T1"]["error"]))

21
12


In [156]:
print(sum(df[df["group"] == "C"]["price"] != 0))
print(sum(df[df["group"] == "T1"]["price"] != 0))


413
403


In [157]:
print(sum(df[df["group"] == "C"]["terminated"]))
print(sum(df[df["group"] == "T1"]["terminated"]))


55
78


In [225]:
print(sum((~df["error"]) & (~df["terminated"])))
print(sum((~df["terminated"]) & (~df["error"])))

0
0


In [226]:
print(sum((~df["error"]) & (~df["terminated"]) & (df['price'] == 0)))
print(sum((~df["terminated"]) & (~df["error"]) & (df["price"] == 0)))


0
0


In [228]:
(~df["error"]) & (~df["terminated"])

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [187]:
(~df["terminated"]) & (~df["error"])

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Length: 1000, dtype: bool

In [204]:
df = df[['terminated', 'error']]

In [233]:
df["terminated"]

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: terminated, Length: 1000, dtype: bool

In [217]:
df["terminated"].astype(bool)

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: terminated, Length: 1000, dtype: bool

In [231]:
df["terminated"]

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: terminated, Length: 1000, dtype: bool

In [223]:
~df["terminated"].astype(bool)

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: terminated, Length: 1000, dtype: bool

In [238]:
df.to_parquet("result.parquet")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 7.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.
